# Applying CNN and RNN to the processed audio files.

In [ ]:
import os
from scipy.io import wavfile
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from keras.layers import Conv2D, MaxPool2D, Flatten, LSTM
from keras.layers import Dropout, Dense, TimeDistributed
from keras.models import Sequential
from keras.utils import to_categorical
from sklearn.utils.class_weight import compute_class_weight
from tqdm import tqdm
from python_speech_features import mfcc
import librosa as lb

In [ ]:
def build_rand_feat():
    X=[]
    y=[]
    _min,_max=float('inf'),-float('inf')
    for _ in tqdm(range(n_samples)):
        rand_class=np.random.choice(class_dist.index,p=prob_dist)
        file=np.random.choice(df[df.label==rand_class].index)
        rate,wav=wavfile.read('clean/'+file)
        
        label=df.at[file,'label']
        rand_index=np.random.randint(0,wav.shape[0]-config.step)
        sample=wav[rand_index:rand_index+config.step]
        X_sample=lb.feature.mfcc(sample,sr=config.rate,n_mfcc=config.nfeat).T
        
        _min=min(np.amin(X_sample),_min)
        _max=max(np.amax(X_sample),_max)
        
        X.append(X_sample if config.mode=='convo' else X_sample.T)
        y.append(classes.index(label))
    X,y=np.array(X),np.array(y)
    X=(X-_min)/(_max-_min)
    
    if config.mode=='convo':
        X=X.reshape(X.shape[0],X.shape[1],X.shape[2],1)
    elif config.mode=='time':
        X=X.reshape(X.shape[0],X.shape[1],X.shape[2])
    y=to_categorical(y,num_classes=10)
    
    return X,y

In [ ]:
Creating CNN model

In [19]:
def get_conv_model():
    model=Sequential()
    model.add(Conv2D(16,(3,3),activation='relu',strides=(1,1),padding='same',input_shape=input_shape))
    model.add(Conv2D(32,(3,3),activation='relu',strides=(1,1),padding='same'))
    model.add(Conv2D(68,(3,3),activation='relu',strides=(1,1),padding='same'))
    model.add(Conv2D(128,(3,3),activation='relu',strides=(1,1),padding='same'))
    model.add(MaxPool2D((2,2)))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(128,activation='relu'))
    model.add(Dense(64,activation='relu'))
    model.add(Dense(10,activation='softmax'))
    model.summary()
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])
    
    return model



class Config():
    def __init__(self,mode='conv',nfilt=26,nfeat=13,nfft=512,rate=16000):
        self.mode=mode
        self.nfilt=nfilt
        self.nfeat=nfeat
        self.nfft=nfft
        self.rate=rate
        self.step=int(rate/10)

df = pd.read_csv('ins.csv')
df.set_index('fname', inplace=True)

for f in df.index:
    rate, signal = wavfile.read('clean/'+f)
    df.at[f, 'length'] = signal.shape[0]/rate

classes = list(np.unique(df.label))
class_dist = df.groupby(['label'])['length'].mean()

n_samples=2*int(df['length'].sum()/0.1)
prob_dist=class_dist/class_dist.sum()
choices=np.random.choice(class_dist.index,p=prob_dist)

fig, ax = plt.subplots()
ax.set_title('Class Distribution', y=1.08)
ax.pie(class_dist, labels=class_dist.index, autopct='%1.1f%%',
       shadow=False, startangle=90)
ax.axis('equal')
plt.show()

config=Config(mode='conv')

#CNN
if config.mode=='conv':
    X,y=build_rand_feat()
    y_flat=np.argmax(y,axis=1)
    input_shape=(X.shape[1],X.shape[2],1)
    model=get_conv_model()
#time means RNN    
elif config.mode=='time':
    X,y=build_rand_feat()
    y_flat=np.argmax(y,axis=1)
    input_shape=(X.shape[1],X.shape[2])
    model=get_conv_model()

    
class_weight=compute_class_weight('balanced',np.unique(y_flat),y_flat)
model.fit(X,y,epochs=10,batch_size=32,shuffle=True,class_weight=class_weight)

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 13, 4, 16)         160       
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 13, 4, 32)         4640      
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 13, 4, 68)         19652     
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 13, 4, 128)        78464     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 6, 2, 128)         0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 6, 2, 128)         0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 1536)             

10208/26410 [==========>...................] - ETA: 42:47 - loss: 2.2979 - acc: 0.18 - ETA: 21:58 - loss: 2.2975 - acc: 0.12 - ETA: 14:56 - loss: 2.2929 - acc: 0.13 - ETA: 11:23 - loss: 2.2843 - acc: 0.13 - ETA: 9:15 - loss: 2.2763 - acc: 0.1500 - ETA: 7:49 - loss: 2.2651 - acc: 0.161 - ETA: 6:48 - loss: 2.2592 - acc: 0.160 - ETA: 6:02 - loss: 2.2380 - acc: 0.164 - ETA: 5:27 - loss: 2.2430 - acc: 0.156 - ETA: 4:58 - loss: 2.2480 - acc: 0.168 - ETA: 4:13 - loss: 2.2522 - acc: 0.164 - ETA: 3:40 - loss: 2.2449 - acc: 0.165 - ETA: 3:17 - loss: 2.2358 - acc: 0.171 - ETA: 3:08 - loss: 2.2358 - acc: 0.171 - ETA: 3:00 - loss: 2.2335 - acc: 0.170 - ETA: 2:52 - loss: 2.2323 - acc: 0.167 - ETA: 2:46 - loss: 2.2320 - acc: 0.168 - ETA: 2:33 - loss: 2.2339 - acc: 0.173 - ETA: 2:23 - loss: 2.2327 - acc: 0.175 - ETA: 2:14 - loss: 2.2276 - acc: 0.175 - ETA: 2:10 - loss: 2.2263 - acc: 0.173 - ETA: 2:07 - loss: 2.2291 - acc: 0.173 - ETA: 2:04 - loss: 2.2261 - acc: 0.173 - ETA: 2:01 - loss: 2.2223 - acc: 

26410/26410 [==============================] - ETA: 6s - loss: 1.5885 - acc: 0.423 - ETA: 5s - loss: 1.5870 - acc: 0.423 - ETA: 5s - loss: 1.5857 - acc: 0.424 - ETA: 5s - loss: 1.5849 - acc: 0.424 - ETA: 5s - loss: 1.5839 - acc: 0.425 - ETA: 5s - loss: 1.5824 - acc: 0.425 - ETA: 5s - loss: 1.5810 - acc: 0.426 - ETA: 5s - loss: 1.5800 - acc: 0.426 - ETA: 5s - loss: 1.5799 - acc: 0.426 - ETA: 5s - loss: 1.5791 - acc: 0.426 - ETA: 5s - loss: 1.5785 - acc: 0.427 - ETA: 5s - loss: 1.5775 - acc: 0.427 - ETA: 4s - loss: 1.5780 - acc: 0.427 - ETA: 4s - loss: 1.5777 - acc: 0.427 - ETA: 4s - loss: 1.5778 - acc: 0.427 - ETA: 4s - loss: 1.5764 - acc: 0.427 - ETA: 4s - loss: 1.5752 - acc: 0.428 - ETA: 4s - loss: 1.5749 - acc: 0.428 - ETA: 4s - loss: 1.5736 - acc: 0.428 - ETA: 4s - loss: 1.5730 - acc: 0.429 - ETA: 4s - loss: 1.5728 - acc: 0.429 - ETA: 4s - loss: 1.5714 - acc: 0.429 - ETA: 4s - loss: 1.5704 - acc: 0.430 - ETA: 4s - loss: 1.5696 - acc: 0.430 - ETA: 4s - loss: 1.5688 - acc: 0.430 - ETA

12640/26410 [=============>................] - ETA: 31s - loss: 0.9428 - acc: 0.59 - ETA: 30s - loss: 1.0031 - acc: 0.61 - ETA: 30s - loss: 0.9615 - acc: 0.65 - ETA: 29s - loss: 1.0696 - acc: 0.60 - ETA: 28s - loss: 1.0670 - acc: 0.62 - ETA: 29s - loss: 1.0778 - acc: 0.63 - ETA: 29s - loss: 1.1086 - acc: 0.61 - ETA: 28s - loss: 1.1073 - acc: 0.61 - ETA: 29s - loss: 1.0847 - acc: 0.61 - ETA: 29s - loss: 1.0807 - acc: 0.61 - ETA: 29s - loss: 1.0652 - acc: 0.61 - ETA: 29s - loss: 1.0566 - acc: 0.62 - ETA: 28s - loss: 1.0539 - acc: 0.61 - ETA: 28s - loss: 1.0500 - acc: 0.61 - ETA: 28s - loss: 1.0435 - acc: 0.61 - ETA: 28s - loss: 1.0333 - acc: 0.61 - ETA: 29s - loss: 1.0430 - acc: 0.61 - ETA: 29s - loss: 1.0442 - acc: 0.61 - ETA: 29s - loss: 1.0460 - acc: 0.61 - ETA: 29s - loss: 1.0387 - acc: 0.62 - ETA: 29s - loss: 1.0378 - acc: 0.62 - ETA: 30s - loss: 1.0399 - acc: 0.62 - ETA: 30s - loss: 1.0335 - acc: 0.62 - ETA: 30s - loss: 1.0359 - acc: 0.62 - ETA: 30s - loss: 1.0355 - acc: 0.62 - ETA

26410/26410 [==============================] - ETA: 0s - loss: 0.9451 - acc: 0.667 - ETA: 0s - loss: 0.9450 - acc: 0.667 - ETA: 0s - loss: 0.9450 - acc: 0.667 - ETA: 0s - loss: 0.9447 - acc: 0.667 - ETA: 0s - loss: 0.9442 - acc: 0.668 - ETA: 0s - loss: 0.9439 - acc: 0.668 - ETA: 0s - loss: 0.9436 - acc: 0.668 - ETA: 0s - loss: 0.9437 - acc: 0.668 - ETA: 0s - loss: 0.9438 - acc: 0.668 - ETA: 0s - loss: 0.9433 - acc: 0.668 - ETA: 0s - loss: 0.9429 - acc: 0.668 - ETA: 0s - loss: 0.9426 - acc: 0.668 - 32s 1ms/step - loss: 0.9426 - acc: 0.6687
Epoch 3/10


...

26410/26410 [==============================] - ETA: 2s - loss: 0.4234 - acc: 0.847 - ETA: 2s - loss: 0.4230 - acc: 0.847 - ETA: 1s - loss: 0.4240 - acc: 0.847 - ETA: 1s - loss: 0.4244 - acc: 0.847 - ETA: 1s - loss: 0.4245 - acc: 0.847 - ETA: 1s - loss: 0.4252 - acc: 0.847 - ETA: 1s - loss: 0.4257 - acc: 0.847 - ETA: 1s - loss: 0.4261 - acc: 0.846 - ETA: 1s - loss: 0.4259 - acc: 0.847 - ETA: 1s - loss: 0.4263 - acc: 0.846 - ETA: 1s - loss: 0.4262 - acc: 0.846 - ETA: 1s - loss: 0.4262 - acc: 0.846 - ETA: 1s - loss: 0.4268 - acc: 0.846 - ETA: 1s - loss: 0.4264 - acc: 0.846 - ETA: 1s - loss: 0.4264 - acc: 0.846 - ETA: 1s - loss: 0.4261 - acc: 0.846 - ETA: 0s - loss: 0.4263 - acc: 0.846 - ETA: 0s - loss: 0.4265 - acc: 0.846 - ETA: 0s - loss: 0.4265 - acc: 0.846 - ETA: 0s - loss: 0.4265 - acc: 0.846 - ETA: 0s - loss: 0.4264 - acc: 0.846 - ETA: 0s - loss: 0.4260 - acc: 0.846 - ETA: 0s - loss: 0.4263 - acc: 0.846 - ETA: 0s - loss: 0.4262 - acc: 0.846 - ETA: 0s - loss: 0.4263 - acc: 0.846 - ETA

12480/26410 [=============>................] - ETA: 35s - loss: 0.3707 - acc: 0.90 - ETA: 33s - loss: 0.4140 - acc: 0.89 - ETA: 33s - loss: 0.3705 - acc: 0.88 - ETA: 34s - loss: 0.3764 - acc: 0.86 - ETA: 33s - loss: 0.4152 - acc: 0.84 - ETA: 33s - loss: 0.4471 - acc: 0.83 - ETA: 32s - loss: 0.4228 - acc: 0.84 - ETA: 33s - loss: 0.4189 - acc: 0.84 - ETA: 33s - loss: 0.4345 - acc: 0.84 - ETA: 33s - loss: 0.4251 - acc: 0.84 - ETA: 33s - loss: 0.4234 - acc: 0.84 - ETA: 32s - loss: 0.4184 - acc: 0.84 - ETA: 32s - loss: 0.4245 - acc: 0.84 - ETA: 32s - loss: 0.4144 - acc: 0.84 - ETA: 32s - loss: 0.4145 - acc: 0.84 - ETA: 32s - loss: 0.4232 - acc: 0.84 - ETA: 31s - loss: 0.4123 - acc: 0.84 - ETA: 31s - loss: 0.4121 - acc: 0.84 - ETA: 31s - loss: 0.4117 - acc: 0.84 - ETA: 31s - loss: 0.4238 - acc: 0.84 - ETA: 31s - loss: 0.4258 - acc: 0.84 - ETA: 31s - loss: 0.4211 - acc: 0.84 - ETA: 31s - loss: 0.4135 - acc: 0.84 - ETA: 31s - loss: 0.4212 - acc: 0.84 - ETA: 31s - loss: 0.4167 - acc: 0.84 - ETA

26410/26410 [==============================] - ETA: 1s - loss: 0.3956 - acc: 0.856 - ETA: 1s - loss: 0.3957 - acc: 0.856 - ETA: 1s - loss: 0.3958 - acc: 0.856 - ETA: 1s - loss: 0.3954 - acc: 0.856 - ETA: 1s - loss: 0.3954 - acc: 0.856 - ETA: 1s - loss: 0.3948 - acc: 0.857 - ETA: 1s - loss: 0.3948 - acc: 0.857 - ETA: 0s - loss: 0.3947 - acc: 0.857 - ETA: 0s - loss: 0.3946 - acc: 0.857 - ETA: 0s - loss: 0.3945 - acc: 0.857 - ETA: 0s - loss: 0.3947 - acc: 0.857 - ETA: 0s - loss: 0.3945 - acc: 0.857 - ETA: 0s - loss: 0.3942 - acc: 0.857 - ETA: 0s - loss: 0.3941 - acc: 0.857 - ETA: 0s - loss: 0.3939 - acc: 0.857 - ETA: 0s - loss: 0.3941 - acc: 0.857 - ETA: 0s - loss: 0.3942 - acc: 0.857 - ETA: 0s - loss: 0.3946 - acc: 0.857 - ETA: 0s - loss: 0.3944 - acc: 0.857 - ETA: 0s - loss: 0.3946 - acc: 0.857 - 32s 1ms/step - loss: 0.3947 - acc: 0.8570
Epoch 10/10


12736/26410 [=============>................] - ETA: 26s - loss: 0.5751 - acc: 0.75 - ETA: 30s - loss: 0.6490 - acc: 0.69 - ETA: 29s - loss: 0.6277 - acc: 0.74 - ETA: 29s - loss: 0.6609 - acc: 0.75 - ETA: 29s - loss: 0.5864 - acc: 0.78 - ETA: 29s - loss: 0.5565 - acc: 0.78 - ETA: 29s - loss: 0.5384 - acc: 0.79 - ETA: 29s - loss: 0.5245 - acc: 0.80 - ETA: 28s - loss: 0.5121 - acc: 0.81 - ETA: 28s - loss: 0.4922 - acc: 0.81 - ETA: 28s - loss: 0.4834 - acc: 0.81 - ETA: 28s - loss: 0.4681 - acc: 0.82 - ETA: 28s - loss: 0.4487 - acc: 0.83 - ETA: 29s - loss: 0.4483 - acc: 0.83 - ETA: 29s - loss: 0.4416 - acc: 0.84 - ETA: 28s - loss: 0.4406 - acc: 0.83 - ETA: 28s - loss: 0.4305 - acc: 0.83 - ETA: 28s - loss: 0.4281 - acc: 0.84 - ETA: 28s - loss: 0.4217 - acc: 0.84 - ETA: 28s - loss: 0.4229 - acc: 0.84 - ETA: 28s - loss: 0.4156 - acc: 0.85 - ETA: 28s - loss: 0.4095 - acc: 0.85 - ETA: 28s - loss: 0.4071 - acc: 0.85 - ETA: 28s - loss: 0.4066 - acc: 0.85 - ETA: 28s - loss: 0.4086 - acc: 0.85 - ETA

26410/26410 [==============================] - ETA: 1s - loss: 0.3643 - acc: 0.870 - ETA: 1s - loss: 0.3641 - acc: 0.870 - ETA: 1s - loss: 0.3645 - acc: 0.870 - ETA: 1s - loss: 0.3645 - acc: 0.870 - ETA: 0s - loss: 0.3645 - acc: 0.870 - ETA: 0s - loss: 0.3642 - acc: 0.870 - ETA: 0s - loss: 0.3644 - acc: 0.870 - ETA: 0s - loss: 0.3646 - acc: 0.870 - ETA: 0s - loss: 0.3644 - acc: 0.870 - ETA: 0s - loss: 0.3647 - acc: 0.869 - ETA: 0s - loss: 0.3646 - acc: 0.869 - ETA: 0s - loss: 0.3645 - acc: 0.869 - ETA: 0s - loss: 0.3650 - acc: 0.869 - ETA: 0s - loss: 0.3650 - acc: 0.869 - ETA: 0s - loss: 0.3651 - acc: 0.869 - ETA: 0s - loss: 0.3653 - acc: 0.869 - ETA: 0s - loss: 0.3654 - acc: 0.869 - 31s 1ms/step - loss: 0.3654 - acc: 0.8697
